In [0]:
# !pip install newspaper3k
# nltk.download('punkt')

In [0]:
import newspaper
import nltk
from bs4 import BeautifulSoup
import re
from newspaper import Article
import csv
import requests
import pandas as pd
from google.colab import files

In [0]:
def get_first_article_info(url):
  '''scrapes the first article on the frontpage of a day page
  on the nyt website, returns a list with its title, url, 
  description and keywords.'''

  # scrape the front page with beautiful soup
  html = requests.get(url)
  soup = BeautifulSoup(html.text, "html.parser")
  
  # getting the first article by scraping its class
  newsoup = soup.find("div", {"class": "css-10wtrbd"})

  # the url 
  url_ending = newsoup.a['href']
  article_url = 'https://www.nytimes.com' + url_ending
  # the title 
  article_title = newsoup.h2.getText()
  # the article description on the front page
  article_info = newsoup.find('p').getText()

  # using newspaper library to extract keywords
  article = Article(article_url)
  article.download()
  article.parse()
  article.nlp()

  return [article_title, article_url, article_info, article.keywords]

In [0]:
get_first_article_info('https://www.nytimes.com/issue/todayspaper/2020/03/01/todays-new-york-times')

['Winning South Carolina, Biden Makes Case Against Sanders: ‘Win Big or Lose’',
 'https://www.nytimes.com/2020/02/29/us/politics/joe-biden-south-carolina-primary.html',
 'Joseph R. Biden Jr. drew on his decades-long relationships and leveraged his close bond with black voters to wrap up a state long considered his stronghold.',
 ['winning',
  'lose',
  'sanders',
  'big',
  'states',
  'campaign',
  'case',
  'makes',
  'carolina',
  'support',
  'south',
  'biden',
  'mr',
  'voters',
  'win',
  'super']]

In [0]:
# getting urls of all nyt front pages in february
feb_urls = []
for i in range(1, 9):
  feb_urls.append(['https://www.nytimes.com/issue/todayspaper/2020/02/0' + str(i) + '/todays-new-york-times', '2020/02/0' + str(i)])
for i in range(10,28): 
  feb_urls.append(['https://www.nytimes.com/issue/todayspaper/2020/02/' + str(i) + '/todays-new-york-times', '2020/02/0' + str(i)])

In [0]:
def get_month_info(urls):
  lo_info = []
  for url, date in urls:
    info = get_first_article_info(url)
    info.append(date)
    lo_info.append(info)
  return lo_info

In [0]:
feb_news = get_month_info(feb_urls)

In [0]:
# creating df from the info
df = pd.DataFrame(feb_news, columns=['title', 'url','description', 'keywords', 'date']) 
df[:5] 

,title,url,description,keywords,date
0,"Republicans Block Impeachment Witnesses, Clear...",https://www.nytimes.com/2020/01/31/us/politics...,The narrow vote came after Republican senators...,"[republicans, acquittal, trump, clearing, impe...",2020/02/01
1,"While Stained in History, Trump Will Emerge Fr...",https://www.nytimes.com/2020/02/01/us/politics...,"His acquittal in the Senate assured, the embol...","[stained, reelection, emerge, trump, unshackle...",2020/02/02
2,SARS Stung the Global Economy. The Coronavirus...,https://www.nytimes.com/2020/02/03/business/ec...,"In the nearly 20 years since SARS, China’s imp...","[global, economy, sars, coronavirus, holiday, ...",2020/02/03
3,Delays Mar Iowa Caucuses as Democrats Start No...,https://www.nytimes.com/2020/02/03/us/politics...,A “quality control” issue with a new system of...,"[world, dont, reliable, start, iowa, caucuses,...",2020/02/04
4,Trump Claims End of ‘American Decline’ While A...,https://www.nytimes.com/2020/02/05/us/politics...,The interactions between President Trump and S...,"[trump, american, decline, impeachment, nation...",2020/02/05


In [0]:
# save the df as csv file and download it locally
df.to_csv('feb_news_info.csv') 
files.download('feb_news_info.csv')